In [ ]:
!pip install torch transformers datasets accelerate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
import pandas as pd
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM, EarlyStoppingCallback
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from huggingface_hub import login
from transformers import Trainer, TrainingArguments


In [ ]:
hugging_face_token = os.getenv("HF_TOKEN")
login(hugging_face_token)

In [ ]:
# Load the Mistral 7B model and tokenizer
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Read the data from the .txt file
with open('final_document.txt', 'r') as file:
    lines = file.readlines()

def extract_questions_answers(lines):
    questions = []
    answers = []

    # Initialize a variable to keep track of the last question
    current_question = None

    for line in lines:
        line = line.strip()  # Remove any leading/trailing whitespace
        if line.startswith("Question:"):
            current_question = line.replace("Question: ", "").strip()
        elif line.startswith("Answer:"):
            if current_question:  # Ensure we have a current question
                answer = line.replace("Answer: ", "").strip()
                questions.append(current_question)
                answers.append(answer)
                current_question = None  # Reset for the next pair

    return questions, answers
questions, answers = extract_questions_answers(lines)

In [ ]:
data_frame = pd.DataFrame({"questions": questions, "answers": answers})

In [ ]:
dataset = Dataset.from_dict({"questions": questions, "answers": answers})

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.1)  # 20% for testing


In [ ]:
# Access the training and testing datasets
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


In [ ]:
def tokenize_function(examples):
    # Check if the tokenizer has a pad token; if not, set it
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding


    # Combine questions with an instruction
    encodings = tokenizer(
        [f"As a legal assistant, answer the question with references to eu ai act if possible: {q}:" for q in examples['questions']],  # Instruction added
        truncation=True,
        padding='max_length',
        max_length=512,  # Adjust as needed
        return_tensors='pt'
    )

    # Tokenize answers as labels
    answer_encodings = tokenizer(
        examples['answers'],
        truncation=True,
        padding='max_length',
        max_length=512,  # Adjust as needed
        return_tensors='pt'
    )

    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': answer_encodings['input_ids'],  # Answers are the labels
    }

In [ ]:
# Tokenize the datasetrain_datasetts
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1793 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Remove the unwanted columns (questions and answers)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['questions', 'answers'])
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(['questions', 'answers'])

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1793
})

In [ ]:
# Ensure proper formatting (adjust columns as necessary)
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1793
})

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank for the low-rank adaptation
    lora_alpha=8,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    task_type=TaskType.CAUSAL_LM  # Task type
)

In [ ]:
# Prepare the model with LoRA
model = get_peft_model(model, lora_config)

# Move model to device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Lin

In [ ]:
import math

In [ ]:
# Global Parameters
L_RATE = 0.001
BATCH_SIZE = 2
BATCH_SIZE = math.ceil(BATCH_SIZE)
gradient_accumulation_steps = 2
WEIGHT_DECAY = 0.01

STEPS = int(len(tokenized_train_dataset) / (BATCH_SIZE * gradient_accumulation_steps))
number_of_epochs = 4
MAX_STEPS = int(number_of_epochs * STEPS)
WARM_UP_STEPS = int(0.01 * MAX_STEPS)
LOGGING_STEPS = 50
SAVE_STEPS = 200
SAVE_TOTAL_LIM  = 8

In [ ]:
SAVE_STEPS

200

In [ ]:
# Define training arguments
training_args = TrainingArguments(
   output_dir="./results",
   evaluation_strategy="steps",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   #predict_with_generate=True,
   push_to_hub=False,
   gradient_accumulation_steps=gradient_accumulation_steps,
   max_steps=MAX_STEPS,
   warmup_steps=WARM_UP_STEPS,
   logging_steps=LOGGING_STEPS,
   eval_steps=LOGGING_STEPS,
   #generation_max_length=512,
   save_steps=SAVE_STEPS,
   load_best_model_at_end=True,   # Ensures the best model is loaded at the end
   metric_for_best_model="eval_loss",  # Replace with your chosen metric
   greater_is_better=False,  # False for loss minimization
   remove_unused_columns=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer with early stopping
trainer = Trainer(
    model=model,  # Your pre-initialized AutoModelForCausalLM
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Your tokenized training dataset
    eval_dataset=tokenized_eval_dataset,    # Your tokenized evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # Stops if no improvement for 2 eval steps
)


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

Train dataset size: 1793
Eval dataset size: 200


In [ ]:
print(f"Train dataset columns: {tokenized_train_dataset.column_names}")
print(f"Eval dataset columns: {tokenized_eval_dataset.column_names}")

Train dataset columns: ['input_ids', 'attention_mask', 'labels']
Eval dataset columns: ['input_ids', 'attention_mask', 'labels']


In [ ]:
# Start training
trainer.train()

Step,Training Loss,Validation Loss
50,2.815600,1.768169
100,1.675400,1.634731
150,1.525700,1.554325
200,1.543500,1.530122
250,1.613000,1.557900
300,1.617800,1.600568
350,1.675000,1.520855
400,1.504600,1.520271
450,1.597700,1.525349
500,1.530300,1.498447


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=1792, training_loss=1.537861894283976, metrics={'train_runtime': 9150.0799, 'train_samples_per_second': 0.783, 'train_steps_per_second': 0.196, 'total_flos': 1.5658689635549184e+17, 'train_loss': 1.537861894283976, 'epoch': 3.9955406911928653})

In [ ]:
output_dir = "/content/eu_ai_model_mistral7b"
trainer.save_model(output_dir)

In [ ]:
!zip -r saved_model.zip eu_ai_model_mistral7b

  adding: eu_ai_model_mistral7b/ (stored 0%)
  adding: eu_ai_model_mistral7b/adapter_config.json (deflated 52%)
  adding: eu_ai_model_mistral7b/adapter_model.safetensors (deflated 7%)
  adding: eu_ai_model_mistral7b/README.md (deflated 66%)
  adding: eu_ai_model_mistral7b/training_args.bin (deflated 51%)


In [ ]:
trainer.push_to_hub(output_dir)

Upload 8 LFS files:   0%|          | 0/8 [00:00<?, ?it/s]

events.out.tfevents.1727446396.e43ae22af8f2.1867.1:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1727446490.e43ae22af8f2.8258.0:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

events.out.tfevents.1727445979.e43ae22af8f2.1867.0:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

events.out.tfevents.1727448033.e43ae22af8f2.14823.0:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1727448054.e43ae22af8f2.14823.1:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1727448121.e43ae22af8f2.15440.0:   0%|          | 0.00/22.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Orcawise/results/commit/a5b46b219117cbfe847cb87c6562c6b51fc6df4c', commit_message='/content/eu_ai_model_mistral7b', commit_description='', oid='a5b46b219117cbfe847cb87c6562c6b51fc6df4c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import os
import time

def generate_response(prompt, max_length=512):
    """Generate response from the model for a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    """
    With lora config one has to use **kwargs i.e key arguments such as input_ids=inputs["input_ids"]
    """
    outputs = model.generate(
                              input_ids=inputs["input_ids"],
                               max_length=max_length,

        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def real_time_testing(*args):
    """
    Process either a single question or a text file of questions. For a single question, print the response.
    For a text file, save the questions and generated answers to a text file.

    Args:
        *args: Single question as a string or text file path as a string.
    """
    questions = []
    is_single_question = False

    # Check if the input is a single question or a file path
    if len(args) == 1 and isinstance(args[0], str):
        input_arg = args[0]
        if os.path.isfile(input_arg):  # Check if the input is a file path
            txt_filename = input_arg
            # Read questions from text file
            try:
                with open(txt_filename, 'r', encoding='utf-8') as txt_file:
                    questions = [line.strip() for line in txt_file.readlines() if line.strip()]
            except IOError as e:
                print(f"Error reading text file {txt_filename}: {e}")
                return
        else:  # Assume the input is a single question
            questions.append(input_arg)
            is_single_question = True
    else:
        print("Invalid input. Please provide a single question or a text file path.")
        return

    # Initialize variables to hold instruction-output pairs
    instruction_output_pairs = []

    start = time.time()

    # Generate outputs for each question and collect pairs
    for idx, question in enumerate(questions):
        generated_text = generate_response(question)
        instruction_output_pairs.append((question, generated_text))
        if is_single_question:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")
        else:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

    if not is_single_question:
        # Define the output text file path
        output_txt_file = 'generated_outputs.txt'

        # Write instruction-output pairs to text file
        try:
            with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
                for pair in instruction_output_pairs:
                    txt_file.write(f"Question: {pair[0]}\nAnswer: {pair[1]}\n\n")
            print(f"Text file with generated outputs saved: {output_txt_file}")
        except IOError as e:
            print(f"Error writing to text file {output_txt_file}: {e}")

In [ ]:
import pickle

# Load the pickle file
with open('validation_dataset_july12.pkl', 'rb') as file:
    data = pickle.load(file)

# Extract questions from the data
questions = []
for item in data:
    if 'Questions' in item:
        questions.append(item['Questions'])

# Write questions to a text file
with open('Questions.txt', 'w', encoding='utf-8') as file:
    for question in questions:
        file.write(question + '\n')

print("Questions have been extracted and saved to Questions.txt")

Questions have been extracted and saved to Questions.txt


In [ ]:
results

NameError: name 'results' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!zip -r checkpoint_1792.zip /content/results/checkpoint-1792


  adding: content/results/checkpoint-1792/ (stored 0%)
  adding: content/results/checkpoint-1792/adapter_config.json (deflated 52%)
  adding: content/results/checkpoint-1792/optimizer.pt (deflated 8%)
  adding: content/results/checkpoint-1792/adapter_model.safetensors (deflated 7%)
  adding: content/results/checkpoint-1792/scheduler.pt (deflated 56%)
  adding: content/results/checkpoint-1792/README.md (deflated 66%)
  adding: content/results/checkpoint-1792/training_args.bin (deflated 51%)
  adding: content/results/checkpoint-1792/trainer_state.json (deflated 81%)
  adding: content/results/checkpoint-1792/rng_state.pth (deflated 25%)
